In [1]:
def read_file(filepath):
    
    with open (filepath) as f:
        str_text = f.read()
    
    return str_text
    

In [2]:
#read_file('moby_dick_four_chapters.txt')

In [3]:
import spacy

In [4]:
nlp = spacy.load('en_core_web_lg',disable=['parser', 'tagger','ner']) # Disabling so as it processes much faster

In [5]:
nlp.max_length = 1198623

In [6]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [ ]:
d = read_file('moby_dick_four_chapters.txt')